In [141]:
import pandas as pd
import random
from copy import deepcopy
from scipy.linalg import svd, diagsvd
import numpy as np
import matplotlib.pyplot as plt

In [142]:
df = pd.read_csv('data/ratings_small.csv')
df.drop(columns=["timestamp"], inplace=True)
dfp = df.pivot(index=["userId"], columns=["movieId"])
dfp.fillna(2.5, inplace=True)

In [143]:
def get(df, dfp):
    '''Retorna um index aleatório para (user X movie)'''
    ind = [i for i in dfp.index]
    col = [j for i,j in dfp.columns]
    i = df.iloc[random.randint(0,100003)]
    return (ind.index(int(i["userId"])), col.index(int(i["movieId"])))

In [144]:
def predict(df,dfp):
    A = dfp.to_numpy()
    B = deepcopy(A)

    ind = get(df,dfp)
    B[ind[0]][ind[1]] = random.randint(0,5)

    X, Y, Z = svd(B)
    Y[-669:] = 0
    sigma = diagsvd(Y, B.shape[0], B.shape[1])
    B = X @ sigma @ Z
    return ind, A[ind[0]][ind[1]], B[ind[0]][ind[1]]

In [145]:
def multPredictError(df,dfp,i):
    A = dfp.to_numpy()
    B = deepcopy(A)

    indices = []
    for _ in range(i):
        ind = get(df,dfp)
        B[ind[0]][ind[1]] = random.randint(0,5)
        indices.append(ind)

    X, Y, Z = svd(B)
    Y[-650:] = 0
    sigma = diagsvd(Y, B.shape[0], B.shape[1])
    B = X @ sigma @ Z

    error = np.array([abs(A[ind[0]][ind[1]] - B[ind[0]][ind[1]]) for ind in indices])
    return error.mean()

In [146]:
# runs = [1,100,200,300,400,500,600,700,800,900,1000]
# error = [multPredictError(df,dfp,i) for i in runs]
# d = {"Quantidade de dados com ruído": runs, "Média do Erro Absoluto": error}
# print(d)

In [147]:
# runs = [1,100,200,300,400,500,600,700,800,900,1000]
# ruido = 100
# runs = 10
# error = [multPredictError(df,dfp,100) for i in range(10)]
# d = {"Quantidade de dados com ruído": [ruido]*runs, "Média do Erro Absoluto": error}
# print(d)

In [148]:
d = {}
for i in [1,100,200,300,400,500,600,700,800,900,1000]:
    error = [multPredictError(df,dfp,i) for _ in range(1,11)]
    d[i] = error

In [149]:
dfa = pd.DataFrame.from_dict(d)
print(dfa)
dfa.to_csv('dados.csv', index=False)

       1         100       200       300       400       500       600   \
0  1.219566  1.120945  1.201276  1.131674  1.157394  1.076131  1.102688   
1  0.890392  1.071466  1.164892  1.021180  1.090019  1.142438  1.085470   
2  0.150961  1.120828  1.016405  1.073381  1.144646  1.119785  1.108297   
3  1.400374  1.018835  1.122330  1.114171  1.099248  1.154968  1.141791   
4  0.561304  1.126289  1.052876  1.062975  1.093803  1.141661  1.033528   
5  2.397244  1.032176  1.199858  1.155988  1.135792  1.126106  1.062822   
6  2.393772  1.208163  1.096177  1.109915  1.135073  1.154071  1.128228   
7  0.517202  1.340508  1.225339  1.114437  1.073574  1.112532  1.096787   
8  2.097504  1.108655  1.103019  1.090730  1.111155  1.132137  1.113456   
9  1.014039  1.035764  1.115439  1.137904  1.169605  1.109644  1.100796   

       700       800       900       1000  
0  1.118583  1.070425  1.129537  1.104546  
1  1.118521  1.121963  1.114526  1.104787  
2  1.131989  1.107588  1.126308  1.142196 